In [1]:
#import dependancies
import json
from elasticsearch import Elasticsearch, helpers

In [15]:
#initialize Elasticsearch client
client = Elasticsearch()

In [16]:
#query the idex called "pandemics".
client.search(index='pandemics', filter_path=['hits.hits._id', 'hits.hits._type')

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (<ipython-input-16-beb0a48402d7>, line 2)

### Get 10 articles

In [17]:
res = client.search(index="pandemics", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(article_title)s %(section_title)s" % hit["_source"])

Got 401 Hits:
Pandemic Summary
Pandemic Definition
Pandemic Stages
Pandemic Severity
Pandemic Management
Pandemic HIV/AIDS
Pandemic COVID-19
Pandemic Notable outbreaks
Pandemic Cholera
Pandemic Influenza


<ipython-input-17-0f32d2e77005>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = client.search(index="pandemics", body={"query": {"match_all": {}}})


### Create a simple (match) query and extract the results from the nested JSON response.

In [18]:
res = client.search(index="pandemics", 
                      body={"query": 
                            {"match": 
                              {"section_title": "COVID-19"}
                            }
                           })
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(article_title)s %(section_title)s" % hit["_source"])

Got 4 Hits:
Pandemic COVID-19
Science diplomacy and pandemics COVID-19
Disease X COVID-19 (2019–2020)
Superspreading event COVID-19 outbreak 2020


<ipython-input-18-a8986f85655e>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = client.search(index="pandemics",
